# Data Proparation

In this colab we have made all the process to adquire our final dataset that we will use to train the language model generator.

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')
data_path = '/content/drive/Shareddrives/DeepLearning_2022/Final-Project/input/'

Mounted at /content/drive/


## Data Loading

In [ ]:
import os
files = os.listdir(data_path)
print(files)

['Auca-del-senyor-esteve.txt', 'Contes-Andersen.txt', 'Els-herois.txt', 'wiki_data', 'dataset.txt', 'poesia.txt', 'poem_generator.ckpt', 'poem_generator4.ckpt', 'JoanAlcover.txt', 'MariaAngels.txt', 'poem_generator_good.ckpt', 'dataset_v2.txt', 'language_generator.ckpt', 'poesia_v2.txt', 'poem_generator_0.4222.ckpt', 'poesia_v3.txt', 'poem_generator_1.8919loss.ckpt', 'poem_generator_1.8145loss.ckpt', 'poem_generator_1.2897loss.ckpt', 'poem_generator_1.2174loss.ckpt', 'poem_generator_minloss.ckpt', 'llibres_dataset.txt', 'llibres_dataset_v2.txt', 'llibres.txt', 'llibres_dataset_v3.txt', 'roberta_cached_lm_510_llibres_dataset_v3.txt']


In [ ]:
with open(data_path + files[0],"r") as f:
  text = ""
  for line in f:
    text += line
#print(text)

In [ ]:
with open(data_path + files[1],"r") as f:
  text2 = ""
  for line in f:
    text2 += line
#print(text2)

In [ ]:
with open(data_path + files[2],"r") as f:
  text3 = ""
  for line in f:
    text3 += line
#print(text3)

In [ ]:
with open(data_path + "llibres.txt","r") as f:
  text4 = ""
  for line in f:
    text4 += line

In [ ]:
a = text4.split("==")
vuit_llibres = " ".join(a)

In [ ]:
ini = " L'ESTEVET"
end = "FI"
llibre1 = text[text.find(ini):text.find(end)]

In [ ]:
ini = "Joan Cristiá Andersen."
end = "FI"
llibre2 = text2[text2.find(ini):text2.find(end)]

In [ ]:
ini = "PÒRTIC."
end = "FI"
llibre3 = text3[text3.find(ini):text3.find(end)]

In [ ]:
complete = llibre1 + " " + llibre2 + " " + llibre3 + vuit_llibres

## Preprocessing Text

In [ ]:
len(complete)

1954207

In [ ]:
re.sub("[\(\)\"=º]"," ",'(abia))=ºun "amigo')

NameError: ignored

In [ ]:
import re
punt_perc = re.sub("[.;]","%",complete)
punt_vocab = re.sub("[.;]"," ",complete)
voc = preprocessing_text(punt_vocab)
procs = preprocessing_text(punt_perc)
len(procs.split("%"))

23835

In [ ]:
procs

In [ ]:
vocab = vocabulary(voc.split(" "))
print("Total words in the vocabulary",len(vocab))
print("Total words",len(voc.split(" ")))
print("Total phrases",len(procs.split("%")))

Total words in the vocabulary 32116
Total words 360858
Total phrases 23835


In [ ]:
vocab

In [ ]:
#vocab
338224/29330

11.531674053869757

In [ ]:
final = []
for a in procs.split("%"):
  if len(a.split(" ")) <= 1:
    print(a.split(" "))
  else:
    final.append(a)

In [ ]:
len(final)

18771

In [ ]:
import torch
def text_to_seq(text,vocab,add_n = False):
  """
  Given a text split it and a vocabulary, return a list of numbers that refers 
  to the index position of each word in the vocabulary.
  """
  sequence = []
  for word in text:
    if word != "":
      sequence.append(vocab.index(word))
  if add_n:
    sequence.append(vocab.index("\n"))

  return torch.tensor(sequence)

In [ ]:
def data_process(corpus, vocab):
    data = list()
    for text in corpus:
        token_list = text_to_seq(text.split(" "),vocab)
        for i in range(1, len(token_list)):
            n_gram_seq = torch.tensor(token_list[:i+1], dtype=torch.long)
            data.append(n_gram_seq)
    return data

In [ ]:
vocab = vocab[1:]

In [ ]:
train_data = data_process(final, vocab)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [ ]:
X = [i[:-1] for i in train_data]   # taking all the words except the last in the input set
y = [i[-1] for i in train_data]    # taking last words in the output set
print("X shape",len(X))
print("y shape",len(y))
print(X[0],y[0])

X shape 340477
y shape 340477
tensor([8678]) tensor(27816)


In [ ]:
import nltk
import string
import re
def preprocessing_text(text,n = True):
  #To lowecase the text
  result = text.lower()
  #Eliminate digits
  result = re.sub(r'\d+', '', result)
  #Eliminate \n
  result = re.sub("\n\n","",result)
  if n:
    result = re.sub("\n"," ",result)
  #Considering \n as one more word
  else:
    result = re.sub("\n"," \n ",result)
  #Eliminate \t
  #result = re.sub("d'","de ",result)
  #result = re.sub("n'","en ",result)
  #result = re.sub("'m"," em",result)
  #result = re.sub("'l"," el",result)
  #result = re.sub("qu'","que ",result)
  result = re.sub("--","",result)
  result = re.sub("-"," ",result)
  result = re.sub("\t"," ",result)
  #result = re.sub("[íìîï]","i",result)
  #result = re.sub("[óôò]","o",result)
  #result = re.sub("[èéê]","e",result)
  #result = re.sub("[àáâ]","a",result)
  #result = re.sub("[úùü]","u",result)
  result = re.sub("ñ","ny",result)
#['', 'ñ', 'çá', 'î', 'ü', 'ô', 'â', '—', 'á', 'ï', 'ì', 'ê', 'ç', 'ù', '’']
  #Eliminate any punctuation and symbol
  result = re.sub("[\[\]\(\)\"'’=!¡º.,—¿?;:«»*_]",' ',result)
  #Eliminate multiple white spaces
  result = re.sub(' +', ' ', result)

  return result

In [ ]:
processed = preprocessing_text(complete)
#print(processed)

In [ ]:
def vocabulary(words):
  """
  Given a plain text in a list returns a list with all the different words.
  """
  vocab = set()
  for word in words:
    if word not in vocab:
      vocab.add(word)

  return list(vocab)

In [ ]:
vocab = vocabulary(processed.split(" "))
print(vocab[0:20])
print(len(vocab))

15034


In [ ]:
#Decide what to do with words that are not in the vocabulary
def text_to_seq(text,vocab):
  """
  Given a text split it and a vocabulary, return a list of numbers that refers 
  to the index position of each word in the vocabulary.
  """
  sequence = []
  for word in text:
    sequence.append(vocab.index(word))

  return sequence

In [ ]:
a = processed.split(" ")[0:20]
print(text_to_seq(a,vocab))

[0, 7573, 1999, 12308, 13290, 5456, 7487, 3222, 8450, 9692, 13806, 14392, 8791, 9168, 8240, 4584, 7573, 9329, 3522, 7988]


# Wikipedia Data 

In [ ]:
#!ls drive/Shareddrives/DeepLearning_2022/Final-Project/Data/wiki_data/
wiki_data = os.listdir(data_path+"/wiki_data")

In [ ]:
txt = wiki_data[5]+"-plain.txt"
os.listdir(data_path+"/wiki_data/"+wiki_data[5])

['14-Astronomia-pos.xml',
 '14-Astronomia-iulact.txt',
 '14-Astronomia-plain.txt',
 '14-Astronomia.iula',
 '14-Astronomia-seg.xml',
 '14-Astronomia-s.xml']

execution time: 4mins 8 seconds

In [ ]:
complete_text = ""
for folder in wiki_data:
  file_txt = folder+"-plain.txt"
  if os.listdir(data_path+"/wiki_data/"+folder):
    with open(data_path+"wiki_data/"+folder+"/"+file_txt) as f:
      text = ""
      for line in f:
        text += line
    complete_text += " "+text

In [ ]:
processed_complete_text = preprocessing_text(complete_text)

In [ ]:
final_dataset = processed_complete_text + " " + processed

In [ ]:
vocab = vocabulary(final_dataset.split(" "))
print(len(vocab))

41089


In [ ]:
!ls drive/Shareddrives/DeepLearning_2022/Final-Project/Data

Auca-del-senyor-esteve.txt  dataset.txt     wiki_data
Contes-Andersen.txt	    Els-herois.txt


In [ ]:
final_dataset

In [ ]:
with open(data_path+'dataset_v2.txt', 'w') as f:
    f.write(final_dataset)

In [ ]:
with open(data_path+'dataset_transformers.txt', 'w') as f:
    #f.write(final_dataset)
    for l in final:
      f.write(l + "\n")

In [ ]:
len(final_dataset.split(" "))

# Poesia

In [ ]:
with open(data_path + "poesia.txt","r") as f:
  poesia = ""
  for line in f:
    poesia += line


In [ ]:
import random
poesias_list = poesia.split("—")
print(preprocessing_text(poesias_list[random.randint(0,len(poesias_list))],False))

In [ ]:
poesies_proc = []
poesies_vocab = []
for poem in poesias_list:
  poesies_vocab.append(preprocessing_text(poem))
  poesies_proc.append(preprocessing_text(poem,False))

In [ ]:
words = " ".join(poesies_vocab).split(" ")
vocab = vocabulary(words)
print("Number of poems",len(poesies_vocab))
print("Number of words in total",len(words))
print("Number of words of the vocabulary",len(vocab))

Number of poems 84
Number of words in total 10853
Number of words of the vocabulary 3327


In [ ]:
print(vocab)